In [ ]:
# PYTHON CODE

import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

from IPython.core.display import display, HTML
display(HTML(
    '<script>'
        'var waitForPlotly = setInterval( function() {'
            'if( typeof(window.Plotly) !== "undefined" ){'
                'MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });'
                'MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);'
                'clearInterval(waitForPlotly);'
            '}}, 250 );'
    '</script>'
))

display(HTML(
    '<style type="text/css">'
    '.plotly-graph-div {'
        'display: block;'
        'margin-left: auto;'
        'margin-right: auto;'
    '}'
    '</style>'
))


<center><h1 style="font-family:timesnewroman;font-size:40px">T<sub>1</sub> Mapping: Inversion Recovery</h1></center>
<p>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Widely considered the gold standard for T<sub>1</sub> mapping, the inversion recovery technique estimates T<sub>1</sub> values by fitting the signal recovery curve acquired at different delays after an inversion pulse (180°). In a typical inversion recovery experiment (<a href="#fig1">Figure 1</a>), magnetization at thermal equilibrium is inverted using a 180° RF pulse. After the longitudinal magnetization recovers through spin-lattice relaxation for predetermined delay (“inversion time”, TI), a 90° excitation pulse is applied, followed by a readout imaging sequence (typically a spin-echo or gradient-echo readout) to create a snapshot of the longitudinal magnetization state at that TI. Inversion recovery was first developed for NMR in the 1940s (Hahn 1949; Drain 1949), and the first T<sub>1</sub> map was acquired using a saturation-recovery technique (90° as a preparation pulse instead of 180°) by (Pykett and Mansfield 1978). Some distinct advantages of inversion recovery is its large potential range of signal change (up to 2M<sub>0</sub>) and an insensitivity to pulse sequence parameter imperfections (Stikov et al. 2015). Despite its proven robustness at measuring T<sub>1</sub>, inversion recovery is scarcely used in practice, because conventional implementations requires repetition times (TRs) on the order of 2 to 5 T<sub>1</sub> (Steen et al. 1994), making it challenging to acquire whole-organ T<sub>1</sub> maps in a clinically feasible time. Nonetheless, it is continuously used as a reference measurement during the development of new techniques, or when comparing different T<sub>1</sub> mapping techniques, and several variations of the inversion recovery technique have been developed, making it practical for some applications.
</p>

<center style="line-height:1;font-family:timesnewroman;font-size:16px;">
<b>
<a name="fig1" style="color:black;">Figure 1.</a>  Pulse sequence of an inversion recovery experiment.
</b>
</center>

<p>
<center><img src="ir_pulsesequences.png" style="width:640px;height:auto;"></center>

<center> <h2 style="font-family:timesnewroman;font-size:30px">Signal Modelling</h2> </center>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
The steady-state longitudinal magnetization of an inversion recovery experiment can be derived from the Bloch equations for the pulse sequence {θ<sub>180</sub> – TI – θ<sub>90</sub> – (TR-TI)}, and is given by:

</p>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px;margin-top:60px;margin-bottom:60px;">
<b>
$$M_z(TI) =  M_0\frac{1-\cos(\theta_{180})e^{-\frac{TR}{T_1}}-[1-\cos(\theta_{180})]e^{-\frac{TI}{T_1}}}{1-\cos(\theta_{180})\cos(\theta_{90})e^{-\frac{TR}{T_1}}} \mathbf{\tag{1}} $$
</b>
</p>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
where M<sub>z</sub> is the longitudinal magnetization prior to the θ<sub>90</sub> pulse. If the in-phase real signal is desired, it can be calculated by multiplying Eq. 1 by <i>k</i>·sin(θ<sub>90</sub>)·exp(-TE/T2), where <i>k</i> is a signal constant. This general equation can be simplified by grouping together the constants for each measurements regardless of their values (i.e. at each TI, same TE and θ<sub>90</sub> are used) and assuming an ideal inversion pulse:
</p>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px;margin-top:60px;margin-bottom:60px;">
<b>
$$M_z(TI) =  C(1-2e^{-\frac{TI}{T_1}}+e^{-\frac{TR}{T_1}})  \mathbf{\tag{2}} $$
</b>
</p>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
where the first three terms and the denominator of Eq. 1 have been grouped together into the constant C. If the experiment is designed such that TR is long enough to allow for full relaxation of the magnetization (TR > 5T<sub>1</sub>), we can do an additional approximation by dropping the last term in Eq. 2:
</p>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px;margin-top:60px;margin-bottom:60px;">
<b>
$$M_z(TI) =  C(1-2e^{-\frac{TI}{T_1}}) \mathbf{\tag{3}}$$
</b>
</p>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
The simplicity of the signal model described by Eq. 3, both in its equation and experimental implementation, has made it the most widely used equation to describe the signal evolution in an inversion recovery T<sub>1</sub> mapping experiment. The magnetization curves are plotted in <a href="#fig2">Figure 2</a> for near T<sub>1</sub> values of three different tissues in the brain. Note that in many practical implementations, magnitude-only images are acquired, so the signal measured would be proportional to the absolute value of Eq. 3.
</p>

In [ ]:
%% MATLAB/OCTAVE CODE

cd ../qMRLab
startup

In [ ]:
%% MATLAB/OCTAVE CODE

clear all

%% Setup parameters
% All times are in seconds
% All flip angles are in degrees

params.TR = 5.0;
params.TI = linspace(0.001, params.TR, 1000);
            
params.TE = 0.004;
params.T2 = 0.040;
            
params.EXC_FA = 90;
params.INV_FA = 180;

params.signalConstant = 1;

%% Calculate signals
%

% White matter
params.T1 = 0.900;
signal_WM = inversion_recovery.analytical_solution(params, 'GRE-IR', 4);

% Grey matter
params.T1 = 1.500;
signal_GM = inversion_recovery.analytical_solution(params, 'GRE-IR', 4);

% CSF
params.T1 = 4.000;
signal_CSF = inversion_recovery.analytical_solution(params, 'GRE-IR', 4);

In [ ]:
%get params --from Octave
%get signal_WM --from Octave
%get signal_GM --from Octave
%get signal_CSF --from Octave

<center style="line-height:1;font-family:timesnewroman;font-size:16px;">
<b>
    <a name="fig2" style="color:black;">Figure 2.</a> Inversion recovery curves (Eq. 2) for three different T<sub>1</sub> values, approximating the major types of tissues in the brain.
</b>
</center>  

In [ ]:
# PYTHON CODE

init_notebook_mode(connected=True)
# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.

wm = go.Scatter(
    x = params["TI"],
    y = signal_WM,
    name = 'T<sub>1</sub> = 0.9 s (White Matter)'
)

gm = go.Scatter(
    x = params["TI"],
    y = signal_GM,
    name = 'T<sub>1</sub> = 1.5 s (Grey Matter)'
)

csf = go.Scatter(
    x = params["TI"],
    y = signal_CSF,
    name = 'T<sub>1</sub> = 4.0 s (Cerebrospinal Fluid)'
)

data = [wm, gm, csf]

layout = go.Layout(
    title='test',
    width=700,
    height=400,
    margin=go.layout.Margin(
        l=150,
        r=5,
        b=75,
        t=0,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.16191064079952971,
            showarrow=False,
            text='Inversion Time – TI (ms)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.4714285714285711,
            showarrow=False,
            text='Long. Magnetization (M<sub>z</sub>)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.6,
        y=0.2,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)


<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Practically, Eq. 1 is the better choice for simulating the signal of an inversion recovery experiment, as the TRs are often chosen to be greater than 5T<sub>1</sub> of the tissue-of-interest, which rarely coincides with the longest T<sub>1</sub> present (e.g. TR may be sufficiently long for white matter, but not for CSF which could also be present in the volume). Equation 3 also assumes ideal inversion pulses, which is rarely the case due to slice profile effects. <a href="#fig3">Figure 3</a> displays the inversion recovery signal magnitude (complete relaxation normalized to 1) of an experiment with TR = 5 and T<sub>1</sub> values ranging between 250 ms to 5 s, calculated using both equations.
<p/>

<center style="line-height:1.5;font-family:timesnewroman;font-size:16px;text-align:justify;margin-left:60px;margin-right:60px">
<b>
<a name="fig3" style="color:black;">Figure 3.</a> Signal recovery curves simulated using Eq. 3 (solid) and Eq. 1 (dotted) with a TR = 5 s for a T<sub>1</sub> values ranging between 250 ms to 5 s.
</b>
</center>

In [ ]:
%% MATLAB/OCTAVE CODE

clear all

%% Setup parameters
% All times are in seconds
% All flip angles are in degrees

params.TR = 5.0;
params.TI = linspace(0.001, params.TR, 1000);
            
params.TE = 0.004;
params.T2 = 0.040;
            
params.EXC_FA = 90;
params.INV_FA = 180;

params.signalConstant = 1;

clear T1range
T1range = 0.25:0.25:5;
for ii = 1:length(T1range)
    params.T1 = T1range(ii);
    
    signal_T1_Eq1{ii} = inversion_recovery.analytical_solution(params, 'GRE-IR', 1);

    signal_T1_Eq3{ii} = inversion_recovery.analytical_solution(params, 'GRE-IR', 4);
end


In [ ]:
%get T1range --from Octave
%get signal_T1_Eq1 --from Octave
%get signal_T1_Eq3 --from Octave

In [ ]:
# PYTHON CODE

init_notebook_mode(connected=True)

data1 = [dict(
        visible = False,
        x = params["TI"],
        y = abs(signal_T1_Eq3[ii]),
        name = '[Eq. 3] – Long TR approximation') for ii in range(len(T1range))]

data1[3]['visible'] = True

data2 = [dict(
        visible = False,
        x = params["TI"],
        y = abs(signal_T1_Eq1[ii]),
        line = dict(
            color = ('rgb(22, 96, 167)'),
            dash = 'dash'),
        name = '[Eq. 1] – General Equation') for ii in range(len(T1range))]

data2[3]['visible'] = True

data = data1 + data2

steps = []
for i in range(len(T1range)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data1)],
        label = str(T1range[i])
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.05,
    active = 3,
    currentvalue = {"prefix": "T1 value (s): "},
    pad = {"t": 50, "b": 10},
    steps = steps
)]

layout = go.Layout(
    width=700,
    height=450,
    margin=go.layout.Margin(
        l=150,
        r=5,
        b=85,
        t=10,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.16191064079952971,
            showarrow=False,
            text='Inversion Time – TI (ms)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.11,
            y=0.4714285714285711,
            showarrow=False,
            text='Signal (magnitude)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        autorange=False,
        range=[0, 5],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0, 1],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.5,
        y=0.5,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

<center> <h2 style="font-family:timesnewroman;font-size:30px">Data Fitting</h2> </center>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Several factors impact the choice of the inversion recovery fitting algorithm.  If only magnitude images are available, then a polarity-inversion is often implemented to restore the non-exponential magnitude curves (<a href="#fig3">Figure 3</a>) into the exponential form (<a href="#fig2">Figure 2</a>). This process is sensitive to noise due to the noise creating a non-zero level at the signal null, and the spacing of the TIs of the acquired data. If phase data is also available, then a phase term must be added to the fitting equation (Barral et al. 2010). Equation 3 must only be used to fit data for the long TR regime (TR > 5·T<sub>1</sub>), which in practice is rarely satisfied for all tissues in subjects.
</p>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Early implementations of inversion recovery fitting algorithms were designed around the computational power available at the time. These included the “null method” (Pykett et al. 1983), assuming that each T<sub>1</sub> value has unique zero-crossings (see <a href="#fig2">Figure 2</a>), and linear fitting of a rearranged version of Eq. 3 on a semi-log plot (Fukushima & Roeder 1981). Nowadays, a non-linear least-squares fitting algorithm (e.g. Levenberg-Marquardt) is more appropriate, and can be applied to either approximate or general forms of the signal model (Eq. 3 or Eq. 1). More recent work (Barral et al. 2010) demonstrated that T<sub>1</sub> maps can also be fitted much quicker (up to 75 times faster than using Levenberg-Marquardt) to fit  Eq. 1 – without a precision penalty – by using a reduced-dimension non-linear least squares (RD-NLS) algorithm. It was demonstrated that the following simplified 5-parameter equation can be sufficient for accurate T<sub>1</sub> mapping:
</p>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px;margin-top:60px;margin-bottom:60px;">
<b>
$$S(TI) =  a+be^{-\frac{TI}{T_1}} \mathbf{\tag{4}}$$
</b>
</p>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
where <i>a</i> and <i>b</i> are complex values. If magnitude-only data is available, a 3-parameter model can be sufficient by taking the absolute value of Eq. 4.  While the RD-NLS algorithms are too complex to be presented here (the reader is referred to the paper, (Barral et al. 2010)),  the code for these algorithms <a href="http://www-mrsrl.stanford.edu/~jbarral/t1map.html">were released open-source</a> along with the original publication, and are also available as a <a href="https://github.com/neuropoly/qMRLab">qMRLab</a> T<sub>1</sub> mapping model. One important thing to note about Eq. 4 is that it is general – no assumption is made about TR – and is thus as robust as Eq. 1 as long as all pulse sequence parameters other than TI are kept constant between each measurement. <a href="#fig4">Figure 4</a> compares simulated data (Eq. 1) using a range of TRs (1.5·T<sub>1</sub> to 5·T<sub>1</sub>) fitted using either RD-NLS & Eq. 4 or a Levenberg-Marquardt fit of Eq. 2.
</p>

<center style="line-height:1.5;font-family:timesnewroman;font-size:16px;text-align:justify;margin-left:60px;margin-right:60px">
<b>
<a name="fig4" style="color:black;">Figure 4.</a> Fitting comparison of simulated data (blue markers) with T<sub>1</sub> = 1 s and TR = 1.5 to 5 s, using fitted using RD-NLS & Eq. 4 (orange) and Levenberg-Marquardt & Eq. 2 (green, long TR approximation).
</b>
</center>

In [ ]:
%% MATLAB/OCTAVE CODE

clear all

%% Setup parameters
% All times are in milliseconds
% All flip angles are in degrees

params.EXC_FA = 90;
params.INV_FA = 180;
params.TI = 50:50:1500;
params.T1 = 1000;
TR = 1500:50:5000;

for ii = 1:length(TR)
    params.TR = TR(ii);
    Mz_analytical(ii,:) = inversion_recovery.analytical_solution(params, 'GRE-IR', 1);
end

TI = params.TI;


x0(1) = 0.5;
x0(2) = 900;
options.Algorithm = 'levenberg-marquardt';

for ii=1:length(TR)
    y = abs(Mz_analytical(ii,:));
    fun = @(x) abs(x(1) .* (1 - 2.*exp(-TI./x(2)))) - y;

    [fitOutput_nlsq(ii,:), resnorm] = lsqnonlin(fun,x0,[],[],options);
    T1_nlsq(ii) = fitOutput_nlsq(ii,2);
end


irObj = inversion_recovery();
irObj.Prot.IRData.Mat = params.TI';

for ii=1:length(TR)

    data.IRData = Mz_analytical(ii,:);

    fitOutput_barral{ii} = irObj.fit(data);

    T1_barral(ii) = fitOutput_barral{ii}.T1;

end

In [ ]:
%get params --from Octave
%get Mz_analytical --from Octave
%get fitOutput_nlsq --from Octave
%get fitOutput_barral --from Octave
%get TR --from Octave

In [ ]:
# PYTHON CODE

init_notebook_mode(connected=True)

data1 = [dict(
        visible = False,
        mode = 'markers',
        x = params["TI"],
        y = abs(np.squeeze(np.asarray(Mz_analytical[ii]))),
        name = 'Simulated data') for ii in range(len(TR))]

data1[10]['visible'] = True

data2 = [dict(
        visible = False,
        mode = 'lines',
        x = params["TI"],
        y = abs(fitOutput_nlsq[ii,0] * (1 - 2*np.exp(-params['TI']/fitOutput_nlsq[ii,1]))),
        name = 'Fitted T1 [C(1-2*exp(-TI/T1))]:' + str(round(fitOutput_nlsq[ii,1])) + ' ms') for ii in range(len(TR))]

data2[10]['visible'] = True

data3 = [dict(
        visible = False,
        mode = 'lines',
        x = params["TI"],
        y = abs((fitOutput_barral[ii]['ra']+fitOutput_barral[ii]['rb']*np.exp(-params['TI']/fitOutput_barral[ii]['T1']))),
        name = 'Fitted T1 [ra+rb*exp(-TI/T1)]:' + str(round(fitOutput_barral[ii]['T1'])) + ' ms') for ii in range(len(TR))]

data3[10]['visible'] = True



data = data1 + data2 + data3

steps = []
for i in range(len(TR)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data1)],
        label = str(TR[i])
        )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.05,
    active = 10,
    currentvalue = {"prefix": "TR value (ms): "},
    pad = {"t": 50, "b": 10},
    steps = steps
)]

layout = go.Layout(
    width=700,
    height=500,
    margin=go.layout.Margin(
        l=150,
        r=5,
        b=85,
        t=10,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.16191064079952971,
            showarrow=False,
            text='Inversion Time – TI (ms)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.11,
            y=0.4714285714285711,
            showarrow=False,
            text='Signal (magnitude)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        autorange=False,
        range=[0, params['TI'][-1]],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0, 1],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.4,
        y=0.9,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

<center> <h2 style="font-family:timesnewroman;font-size:30px">Benefits and Pitfalls</h2> </center>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
The conventional inversion recovery experiment is considered the gold standard T<sub>1</sub> mapping technique for several reasons. A typical protocol has a long TR value and a sufficient number of inversion times for stable fitting (typically 5 or more) covering the range [0, TR]. It offers a wide dynamic range of signals (up to [-<i>k</i>M<sub>0</sub>, <i>k</i>M<sub>0</sub>]), providing a wide range of inversion times where high SNR is available to sample the signal recovery curve (Fukushima 1981). T<sub>1</sub> maps produced by inversion recovery are largely insensitive to inaccuracies in excitation flip angles and imperfect spoiling (Stikov et al. 2015), as all parameters except TI are constant for each measurement and only a single acquisition is performed (at TI) during each TR. One important pulse sequence design consideration is to avoid acquiring at inversion times where the signal for T<sub>1</sub> values of the tissue-of-interest is nulled, as for magnitude images these voxels will be dominated by Rician noise which can negatively impact the fit the under low SNR circumstances (<a href="#fig5">Figure 5</a>). Inversion recovery can also often be acquired using commonly available standard pulse sequences available on most MRI scanners by setting up a customized acquisition protocol, and does not require any additional calibration measurements like a B<sub>1</sub> map.
</p>

<center style="line-height:1.5;font-family:timesnewroman;font-size:16px;text-align:justify;margin-left:60px;margin-right:60px">
<b>
<a name="fig5" style="color:black;">Figure 5.</a> Monte Carlo simulations (mean and standard deviation (STD), blue markers) and fitted T<sub>1</sub> values (mean and STD, red and green respectively) generated for a T<sub>1</sub> value of 900 ms and 5 TI values linearly spaced across the TR (ranging from 1 to 5 s). A bump in T<sub>1</sub> STD occurs near TR = 3000 ms, which coincides with the TR where the second TI is located near a null point for this T<sub>1</sub> value. 
</b>
</center>

In [ ]:
%% MATLAB/OCTAVE CODE

clear all

%% Setup parameters
% All times are in milliseconds
% All flip angles are in degrees

TRrange = 1000:100:5000;

Model = inversion_recovery; 

x = struct;
x.T1 = 900;

Opt.SNR = 25;
Opt.M0 = 1;
Opt.FAinv = 180;
Opt.FAexcite = 90;

for ii = 1:length(TRrange)
    Opt.TR = TRrange(ii);
    Opt.T1 = x.T1;
    TI = linspace(0.05, Opt.TR, 6)';
    Model.Prot.IRData.Mat = [TI];
    [ra,rb] = Model.ComputeRaRb(x,Opt);
    x.rb = rb;
    x.ra = ra;
    for jj = 1:1000
        [FitResult{ii,jj}, noisyData{ii,jj}] = Model.Sim_Single_Voxel_Curve(x,Opt,0); 
        fittedT1(ii,jj) = FitResult{ii,jj}.T1;
        noisyData_array(ii,jj,:) = noisyData{ii,jj}.IRData;
    end
        
    for kk=1:length(TI)
        meanData(ii,kk) = mean(noisyData_array(ii,:,kk));
        stdData(ii,kk) = std(noisyData_array(ii,:,kk));
    end
    
    meanT1(ii) = mean(fittedT1(ii,:));
    stdT1(ii) = std(fittedT1(ii,:));
end

for ii = 1:length(TRrange)
    dataTI(ii,:) = linspace(0.05, TRrange(ii), 6);
end

for ii = 1:length(TRrange)
    dataTI_long(ii,:) = linspace(0.05, TRrange(ii), 500);
    Model.Prot.IRData.Mat = [dataTI_long(ii,:)];
    Opt.TR = TRrange(ii);
    Opt.T1 = x.T1;
    [ra,rb] = Model.ComputeRaRb(x,Opt);
    x.rb = rb;
    x.ra = ra;

    noiselessData(ii,:) = Model.equation(x);
end

In [ ]:
%get TRrange --from Octave
%get meanData --from Octave
%get stdData --from Octave
%get meanT1 --from Octave
%get stdT1 --from Octave
%get dataTI --from Octave
%get dataTI_long --from Octave
%get noiselessData --from Octave

In [ ]:
# PYTHON CODE

init_notebook_mode(connected=True)

data1 = [dict(
        visible = False,
        x = np.squeeze(np.asarray(dataTI[ii,:])),
        y = np.squeeze(np.asarray(meanData[ii,:])),
        error_y=dict(
            type='data',
            color = ('rgb(22, 96, 167)'),
            array=np.squeeze(np.asarray(stdData[ii,:])),
            visible=True
        ),
        line = dict(
            color = ('rgb(22, 96, 167)'),
            dash = 'dot'),
        mode = 'markers',
        name = 'Monte Carlo simulated signal') for ii in range(len(TRrange))]

data1[28]['visible'] = True

data2 = [dict(
        visible = False,
        x = np.squeeze(np.asarray(dataTI_long[ii,:])),
        y = np.squeeze(np.asarray(noiselessData[ii,:])),
        line = dict(
            color = ('rgb(247, 152, 19)'),
            ),
        name = 'Noiseless signal') for ii in range(len(TRrange))]

data2[28]['visible'] = True

data_meanT1 = [dict(
    visible = False,
    x = TRrange,
    y = meanT1,
    name = 'Mean T<sub>1</sub> (s)',
    xaxis='x2',
    yaxis='y2') for ii in range(len(TRrange))]
data_meanT1[15]['visible'] = True

data_stdT1 = [dict(
    visible = False,
    x = TRrange,
    y = stdT1,
    name = 'STD T<sub>1</sub> (s)',
    xaxis='x2',
    yaxis='y3') for ii in range(len(TRrange))]

data_stdT1[28]['visible'] = True

data = data2 + data1 + data_meanT1 + data_stdT1

steps = []
for i in range(len(TRrange)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data1)],
        label = str(TRrange[i])
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.02,
    active = 28,
    currentvalue = {"prefix": "TR value (ms): "},
    pad = {"t": 50, "b": 10},
    steps = steps
)]

layout = go.Layout(
    margin=go.layout.Margin(
        l=150,
        r=5,
        b=100,
        t=10,
        pad=0
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.16191064079952971,
            showarrow=False,
            text='Inversion Time – TI (ms)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.11,
            y=0.4714285714285711,
            showarrow=False,
            text='Signal (magnitude)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.75,
            y=0.75,
            showarrow=False,
            text='<b>TR (ms)<b>',
            font=dict(
                family='Times New Roman',
                size=14
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.43,
            y=0.35,
            showarrow=False,
            text='<b>Mean T<sub>1</sub> (ms)<b>',
            font=dict(
                family='Times New Roman',
                size=14
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.98,
            y=0.35,
            showarrow=False,
            text='<b>STD T<sub>1</sub> (ms)<b>',
            font=dict(
                family='Times New Roman',
                size=14
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        )
    ],
    xaxis=dict(
        autorange=False,
        range=[0, 5000],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0, 1],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    xaxis2=dict(
        domain=[0.5, 0.90],
        anchor='y2',
        mirror = True,
        side='top',
        ticks='inside',
        showline=True,
    ),
    yaxis2=dict(
        autorange=False,
        range=[500, 1300],
        domain=[0.05, 0.65],
        anchor='x2',
        mirror = True,
        ticks='inside',
        showline=True,
    ),
    yaxis3=dict(
        autorange=False,
        range=[0, 190],
        domain=[0.05, 0.65],
        anchor='x2',
        overlaying='y2',
        side='right',
        ticks='inside',
    ),
    legend=dict(
        x=1.02,
        y=1,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Despite a widely acknowledged robustness for measuring accurate T<sub>1</sub> maps, inversion recovery is not often used in studies. An important drawback of this technique is the need for long TR values, generally on the order of a few T<sub>1</sub> for general models (e.g. Eqs. 1 and 4), and up to 5·T<sub>1</sub> for long TR approximated models (Eq. 3). It takes about to 10-25 minutes to acquire a single-slice T<sub>1</sub> map using the inversion recovery technique, as only one TI is acquired per TR  (2-5 s) and conventional cartesian gradient readout imaging acquires one phase encode line per excitation (~100-200). The long acquisition time makes it challenging to acquire whole-organ T<sub>1</sub> maps in clinically feasible protocol times. Nonetheless, it is useful as a reference measurement for comparisons against other T<sub>1</sub> mapping methods, or to acquire a single-slice T<sub>1</sub> map of a tissue to get T<sub>1</sub> estimates for optimization of other pulse sequences.
</p>

<center> <h2 style="font-family:timesnewroman;font-size:30px">Other Saturation-Recovery T<sub>1</sub> Mapping techniques</h2> </center>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Several variations of the inversion recovery pulse sequence were developed to overcome challenges like those specified above. Amongst them, the Look-Locker technique (Look and Locker 1970) stands out as one of the most widely used in practice. Instead of a single 90° acquisition per TR, a periodic train of small excitation pulses θ are applied after the inversion pulse, {θ<sub>180</sub> – 𝛕 – θ – 𝛕 – θ – ...}, where  𝛕 = TR/n and n is the number of sampling acquisitions. This pulse sequence samples the inversion time relaxation curve much more efficiently than conventional inversion recovery, but at a cost of lower SNR. However, because the magnetization state of each TI measurement depends on the previous series of θ excitation, it has higher sensitivity to B<sub>1</sub>-inhomogeneities and imperfect spoiling compared to inversion recovery (Gai et al. 2013; Stikov et al. 2015) . Nonetheless, Look-Locker is widely used for rapid T<sub>1</sub> mapping applications, and variants like MOLLI (Modified Look-Locker Inversion recovery) are widely used for cardiac T<sub>1</sub> mapping (Messroghli et al. 2004).
</p>

<p style="line-height:2;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Another inversion recovery variant that’s worth mentioning is saturation recovery, in which the inversion pulse is replaced with a saturation pulse: {θ<sub>90</sub> – TI – θ<sub>90</sub>}. This technique was used to acquire the very first T<sub>1</sub> map (Pykett and Mansfield 1978). Unlike inversion recovery, this pulse sequence does not need a long TR to recover to its initial condition; every θ<sub>90</sub> pulse resets the longitudinal magnetization to the same initial state. However, to properly sample the recovery curve, TIs still need to reach the order of ~T<sub>1</sub>, the dynamic range of signal potential is cut in half ([0, M<sub>0</sub>]), and the short TIs (which have the fastest acquisition times) have the lowest SNRs.
</p>

<p style="line-height:1.25;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px;color:#696969">
<b><i>
This blog post is powered by <a href="https://github.com/neuropoly/qMRLab">qMRLab</a>. To view the code that was used to generate the figures in this blog post, hover your cursor in the top left of the page and click "All cells" in the "Display contents" popup that appears.
</i></b>

<p style="line-height:1.25;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px;color:#696969">
<b><i>
An interactive Jupyter Notebook version of this blog post is also available throught MyBinder, and can be viewed <a href="https://mybinder.org/v2/gh/qMRLab/t1_notebooks/irblog?filepath=ir_blog%2FInversionRecovery.ipynb">here</a>
</i></b>
</p>

<center> <h2 style="font-family:timesnewroman;font-size:30px">Works Cited</h2> </center>

<p style="line-height:1.5;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Barral JK, Gudmundson E, Stikov N, et al. (2010) A robust methodology for in vivo T<sub>1</sub> mapping. <i>Magn. Reson. Med.</i> 64(4): 1057–1067.
</p>

<p style="line-height:1.5;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Drain LE (1949) A Direct Method of Measuring Nuclear Spin-Lattice Relaxation Times. <i>Proceedings of the Physical Society. Section A</i> 62(5): 301–306.
</p>

<p style="line-height:1.5;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
    Fukushima, E. & Roeder, S., 1981. <i>Experimental Pulse NMR. A Nuts and Bolts Approach</i>, Reading, Massachusetts : Addison-Wesley Publ. Comp., Inc.
</p>

<p style="line-height:1.5;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Gai, N.D. et al., 2013. Modified Look-Locker T<sub>1</sub> evaluation using Bloch simulations: human and phantom validation. <i>Magn. Reson. Med.</i>, 69(2), pp.329–336.
</p>

<p style="line-height:1.5;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Hahn, E.L., 1949. An Accurate Nuclear Magnetic Resonance Method for Measuring Spin-Lattice Relaxation Times. <i>Physics Review</i>, 76(1), pp.145–146.
</p>

<p style="line-height:1.5;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Look, D.C. & Locker, D.R., 1970. Time Saving in Measurement of NMR and EPR Relaxation Times. <i>The Review of scientific instruments</i>, 41(2), pp.250–251.
</p>

<p style="line-height:1.5;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Messroghli, D.R. et al., 2004. Modified Look-Locker inversion recovery (MOLLI) for high-resolution T<sub>1</sub> mapping of the heart. <i>Magn. Reson. Med.</i>, 52(1), pp.141–146.
</p>

<p style="line-height:1.5;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Pykett, I.L. et al., 1983. Measurement of spin-lattice relaxation times in nuclear magnetic resonance imaging. <i>Physics in medicine and biology</i>, 28(6), pp.723–729.
</p>

<p style="line-height:1.5;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Pykett, I.L. & Mansfield, P., 1978. A line scan image study of a tumorous rat leg by NMR. <i>Physics in medicine and biology</i>, 23(5), pp.961–967.
</p>

<p style="line-height:1.5;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
Steen, R.G. et al., 1994. Precise and accurate measurement of proton T<sub>1</sub> in human brain in vivo: validation and preliminary clinical application. <i>J. Magn. Reson. Imaging</i>, 4(5), pp.681–691.
</p>

<p style="line-height:1.5;font-family:timesnewroman;font-size:18px;text-align:justify;margin-left:60px;margin-right:60px">
    Stikov, N. et al., 2015. On the accuracy of T<sub>1</sub> mapping: Searching for common ground. <i>Magn. Reson. Med.</i>, 73(2), pp.514–522.
</p>
